In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
to_run = True
if to_run:
    !git clone https://github.com/nackta/text_classification.git # /content/drive/[your path]
    # /content/drive/[your path]
    %cd text_classification
    to_run = False
    # !wandb login

In [ ]:
!pip install transformers

In [ ]:
import os
import pickle

import pandas as pd
import numpy as np

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

from dataset import *
from trainer import Trainer

In [ ]:
open_file = open(os.path.join("./data/train_pos.pkl"), "rb")
train_pos = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/train_neg.pkl"), "rb")
train_neg = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/dev_pos.pkl"), "rb")
dev_pos = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/dev_neg.pkl"), "rb")
dev_neg = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/test.pkl"), "rb")
test = pickle.load(open_file)
open_file.close()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)
test_dataset = SentimentTestDataset(tokenizer, test)

train_batch_size=128
eval_batch_size=64
test_batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)

dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)
learning_rate = 5e-5

In [ ]:
trainer = Trainer(train_loader=train_loader, dev_loader=dev_loader, test_loader=test_loader, model=model, lr=learning_rate, optimizer=AdamW, device=device)

In [ ]:
trainer.train(train_epoch=1)

In [ ]:
predictions=trainer.test()

In [ ]:
if not os.path.isdir("submission"):
    os.mkdir("submission")
test_df = pd.read_csv('./data/test_no_label.csv')
test_df['Category'] = predictions
test_df.to_csv('./submission/submission_test.csv', index=False)